In [1]:
import folium
import os
import pandas as pd
import numpy as np
import glob
import csv
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

plt.style.use('bmh')

### read files into a dataframe

In [2]:
#reading the files:
DATA_DIR = "./nexar/"
!ls nexar

f1  f2	high_rated-2018-11-24  zero_rated-2018-11-24


In [3]:
samples_accel = glob.glob("nexar/*/*/*_Accel.csv")
samples_gyro=glob.glob("nexar/*/*/*_Gyro.csv")
samples_gps=glob.glob("nexar/*/*/*_GPS.csv")
print("there are {}  accel samples".format(len(samples_accel)))
print("there are {}  gyro samples".format(len(samples_gyro)))
print("there are {}  GPS samples".format(len(samples_gps)))
samples_gyro[:5], samples_accel[:5],samples_gps[:5]


there are 4133  accel samples
there are 4010  gyro samples
there are 4138  GPS samples


(['nexar/zero_rated-2018-11-24/01664c5ac370136abc20d7ac52d0ec48/01664c5ac370136abc20d7ac52d0ec48_Gyro.csv',
  'nexar/zero_rated-2018-11-24/0899c7acdc74c1d2fc16b3ba7dff47c6/0899c7acdc74c1d2fc16b3ba7dff47c6_Gyro.csv',
  'nexar/zero_rated-2018-11-24/049b7df2a36aad19543151765819a213/049b7df2a36aad19543151765819a213_Gyro.csv',
  'nexar/zero_rated-2018-11-24/0515cba67277eed2d285e802421ba4da/0515cba67277eed2d285e802421ba4da_Gyro.csv',
  'nexar/zero_rated-2018-11-24/02680761599287696ed79852c0dd620a/02680761599287696ed79852c0dd620a_Gyro.csv'],
 ['nexar/zero_rated-2018-11-24/01664c5ac370136abc20d7ac52d0ec48/01664c5ac370136abc20d7ac52d0ec48_Accel.csv',
  'nexar/zero_rated-2018-11-24/0899c7acdc74c1d2fc16b3ba7dff47c6/0899c7acdc74c1d2fc16b3ba7dff47c6_Accel.csv',
  'nexar/zero_rated-2018-11-24/049b7df2a36aad19543151765819a213/049b7df2a36aad19543151765819a213_Accel.csv',
  'nexar/zero_rated-2018-11-24/0515cba67277eed2d285e802421ba4da/0515cba67277eed2d285e802421ba4da_Accel.csv',
  'nexar/zero_rated-201

In [5]:
def get_ids(fname):
    """Calculate `sample_id` and short filename."""
    start_dir, filename = os.path.split(fname)
    base_dir, sample_id = os.path.split(start_dir)
    _, sample_class = os.path.split(base_dir)
    sample_class= sample_class
    return (sample_id, sample_class, filename)


## Read file GPS example

In [6]:
#Let's read 1 file of gyro and 1 files of accelerometer:
gps_df= pd.read_csv("nexar/zero_rated-2018-11-24/01664c5ac370136abc20d7ac52d0ec48/01664c5ac370136abc20d7ac52d0ec48_GPS.csv")
gps_df["timestamp"] = pd.to_datetime(gps_df['timestamp'], unit="s")
gps_df=gps_df.resample('1S', on="timestamp").mean()



In [39]:
print(gps_df.shape)
print(gps_df[gps_df.isnull().any(axis=1)].shape)
gps_df.head(20)

(40, 7)
(0, 7)


,longitude,latitude,horizontal_accuracy,vertical_accuracy,altitude,speed,course
timestamp,,,,,,,
2018-10-19 12:40:58,-87.476144,41.570532,5.360,0.0,146.113159,5.15,56.700001
2018-10-19 12:40:59,-87.476101,41.570535,6.432,0.0,147.197327,5.23,61.500000
2018-10-19 12:41:00,-87.476043,41.570565,8.576,0.0,146.460571,5.79,61.700001
2018-10-19 12:41:01,-87.475960,41.570619,6.432,0.0,146.643921,7.58,56.000000
2018-10-19 12:41:02,-87.475856,41.570676,6.432,0.0,147.735413,8.20,53.299999
2018-10-19 12:41:03,-87.475782,41.570712,6.432,0.0,146.105225,9.05,52.299999
2018-10-19 12:41:04,-87.475669,41.570770,5.360,0.0,145.095703,10.03,50.799999
2018-10-19 12:41:05,-87.475557,41.570842,5.360,0.0,145.329407,10.98,49.599998
2018-10-19 12:41:06,-87.475460,41.570898,4.288,0.0,144.785767,11.46,49.900002


In [8]:
files_gps=pd.DataFrame([get_ids(sample) for sample in samples_gps],
                     columns=["sample_id", "class", "gps_file"])

In [9]:
print("Gps: number of dangerous rides {}".format(files_gps[files_gps['class']=="high_rated-2018-11-24"].shape[0]))
print("Gps: number of not dangerous rides {}".format(files_gps[files_gps['class']=="zero_rated-2018-11-24"].shape[0]))
print("Gps:number of files: {}".format(files_gps.shape[0]))

Gps: number of dangerous rides 2223
Gps: number of not dangerous rides 1915
Gps:number of files: 4138


In [10]:
files_gps['class_number']= files_gps['class'].map({"zero_rated-2018-11-24":0,"high_rated-2018-11-24":1})
files_gps.head()


,sample_id,class,gps_file,class_number
0,01664c5ac370136abc20d7ac52d0ec48,zero_rated-2018-11-24,01664c5ac370136abc20d7ac52d0ec48_GPS.csv,0
1,0899c7acdc74c1d2fc16b3ba7dff47c6,zero_rated-2018-11-24,0899c7acdc74c1d2fc16b3ba7dff47c6_GPS.csv,0
2,049b7df2a36aad19543151765819a213,zero_rated-2018-11-24,049b7df2a36aad19543151765819a213_GPS.csv,0
3,0515cba67277eed2d285e802421ba4da,zero_rated-2018-11-24,0515cba67277eed2d285e802421ba4da_GPS.csv,0
4,02680761599287696ed79852c0dd620a,zero_rated-2018-11-24,02680761599287696ed79852c0dd620a_GPS.csv,0


In [11]:
def get_features(base_dir, sample_class, sample_id, filename):
    """Read single file and calculate features."""
    
    # Reading
    fullname = os.path.join(base_dir, sample_id ,sample_class, filename)
    sample = pd.read_csv(fullname)

    # Handling timestamps and spacing
    sample["timestamp"] = pd.to_datetime(sample['timestamp'], unit="s")
    sample = sample.resample("S", on="timestamp").mean()
    
    # Acceleration
    sample["acceleration"] = np.sqrt(np.square(sample).sum(axis=1))
    
    # Statistical aggregates

    # Note the trickery to get right format from pandas
    # without manual renaming
    aggs = (sample
            .apply(["mean", "std", "min", "max"])
            .unstack()
            .reset_index())
    aggs["feature"] = aggs["level_0"].str.cat(aggs["level_1"], sep="_")

    return aggs.set_index("feature")[0]

In [12]:
gps_features=files_gps.apply(lambda f:get_features(DATA_DIR, *f[['sample_id','class','gps_file']].tolist()),axis=1)

In [4]:
gps_features.head(20)

NameError: name 'gps_features' is not defined

## visualisation of GPS data: each ride(~40s) has an avg. long and lat

In [14]:
!python3 -V


Python 3.5.3


In [16]:
!rm -r folium

In [27]:

# !git clone https://github.com/python-visualization/folium
!cd folium
!pwd
# !conda install folium -c conda-forge



# !python3 setup.py install
# imp.find_module('numpy')

/home/ms_rachel_shalom


In [17]:
!sudo pip3 install folium

In [26]:
!python -m pip install cufflinks plotly


In [18]:
# !ls /usr/local/lib/python3.5/dist-packages
# import plotly
!ls /usr/local/lib/python3.5/site-packages/

daal	    numpy		    scikit_learn-0.19.2.dist-info  sklearn
mkl_fft     numpy-1.15.1.dist-info  scipy			   tbb
mkl_random  __pycache__		    scipy-1.1.0.dist-info	   TBB.py


In [12]:
# import catboost

/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

compiletime version 3.5 of module '_catboost' does not match runtime version 3.7



In [14]:
# !ls -la /home/rache/.conda/pkgs


# m = folium.Map(location=[40.693943, -73.985880], control_scale=True, zoom_start=12)

AttributeError: module 'folium' has no attribute 'Map'

In [32]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='rachel_101', api_key='qeDM0gqqLCOIiWP4O0MJ')

In [33]:
plotly.__version__

'3.8.1'

In [34]:
index1=files_gps[files_gps['class_number'] == 1].index
index0=files_gps[files_gps['class_number'] == 0].index
print(gps_features.loc[index1,:].shape)
print(gps_features.loc[index0,:].shape)
files_gps.head()

(2223, 32)
(1915, 32)


,sample_id,class,gps_file,class_number
0,01664c5ac370136abc20d7ac52d0ec48,zero_rated-2018-11-24,01664c5ac370136abc20d7ac52d0ec48_GPS.csv,0
1,0899c7acdc74c1d2fc16b3ba7dff47c6,zero_rated-2018-11-24,0899c7acdc74c1d2fc16b3ba7dff47c6_GPS.csv,0
2,049b7df2a36aad19543151765819a213,zero_rated-2018-11-24,049b7df2a36aad19543151765819a213_GPS.csv,0
3,0515cba67277eed2d285e802421ba4da,zero_rated-2018-11-24,0515cba67277eed2d285e802421ba4da_GPS.csv,0
4,02680761599287696ed79852c0dd620a,zero_rated-2018-11-24,02680761599287696ed79852c0dd620a_GPS.csv,0


In [35]:
mapbox_access_token='pk.eyJ1IjoicmFjaGVsc2hhbG9tIiwiYSI6ImNqaXp3bG1hejBhc2MzcW1odmtqbTQ3Y3IifQ.ygRfsJVddbhT65JnLaBMqA'
site_lat = gps_features['latitude_mean']
site_lon = gps_features['longitude_mean']
locations_name = files_gps['class_number']
data = []
event_types=[1,0]
for event in event_types:
    event_data = dict(
            lat = gps_features.loc[files_gps[files_gps['class_number'] == event].index,'latitude_mean'],
            lon = gps_features.loc[files_gps[files_gps['class_number'] == event].index,'longitude_mean'],
            name = event,
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(event_data)
layout = dict(
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 0, b = 0, l = 0, r = 0),
    font = dict(color = '#FFFFFF', size = 11),
    paper_bgcolor = '#000000',
    mapbox = dict(
        # here you need the token from Mapbox
        accesstoken = mapbox_access_token,
        bearing = 0,
        # where we want the map to be centered
        center = dict(
            lat = 38,
            lon = -94
        ),
        # we want the map to be "parallel" to our screen, with no angle
        pitch = 0,
        # default level of zoom
        zoom = 3,
        # default map style
        style = 'dark'
    )
)



In [36]:
annotations = [dict(
  
              # text I want to display. I used <br> to break it into two lines
              text = 'Rides in the USA dangerous and not dangerous', 
              
              # font and border characteristics
              font = dict(color = '#FFFFFF', size = 14), borderpad = 10, 
              
              # positional arguments
              x = 0.05, y = 0.05, xref = 'paper', yref = 'paper', align = 'left', 
              
              # don't show arrow and set background color
              showarrow = False, bgcolor = 'black'
              )]

# assigning the annotations to the layout
layout['annotations'] = annotations

In [37]:
updatemenus=list([
    # drop-down : select type of event to visualize
    dict(
         # for each button I specify which dictionaries of my data list I want to visualize. Remember I have 7 different
         # types of storms but I have 8 options: the first will show all of them, while from the second to the last option, only
         # one type at the time will be shown on the map
         buttons=list([
            dict(label = 'All ride Events',
                 method = 'update',
                 args = [{'visible': [True, True, True]}]),
            dict(label = 'dangerous',
                 method = 'update',
                 args = [{'visible': [True, False, False]}]),
            dict(label = 'non denagerous',
                 method = 'update',
                 args = [{'visible': [False, True, False]}])
          
        ]),
        # direction where the drop-down expands when opened
        direction = 'down',
        # positional arguments
        x = 0.01,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        # fonts and border
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])

# assign the list of dictionaries to the layout dictionary
layout['updatemenus'] = updatemenus

In [38]:
layout['title'] = 'Ride Events'
figure = dict(data = data, layout = layout)
py.iplot(figure, filename = 'Rachel')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [28]:
import sys
sys.path

['/home/ms_rachel_shalom',
 '/opt/anaconda3/lib/python37.zip',
 '/opt/anaconda3/lib/python3.7',
 '/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/opt/anaconda3/lib/python3.7/site-packages',
 '/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython']

In [4]:
!ls /opt/anaconda3/lib/python3.7/site-packages

alabaster
alabaster-0.7.12.dist-info
anaconda_client-1.7.2-py3.7.egg-info
anaconda_navigator
anaconda_navigator-1.9.6-py3.7.egg-info
anaconda_project
anaconda_project-0.8.2-py3.7.egg-info
apiclient
asn1crypto
asn1crypto-0.24.0-py3.7.egg-info
astroid
astroid-2.1.0.dist-info
astropy
astropy-3.1-py3.7.egg-info
atomicwrites
atomicwrites-1.2.1-py3.7.egg-info
attr
attrs-18.2.0-py3.7.egg-info
babel
Babel-2.6.0-py3.7.egg-info
backcall
backcall-0.1.0.dist-info
backports
backports.os-0.1.1-py3.7.egg-info
backports.shutil_get_terminal_size-1.0.0.dist-info
bcolz
bcolz-1.2.1.dist-info
beautifulsoup4-4.6.3-py3.7.egg-info
binstar_client
bitarray
bitarray-0.8.3-py3.7.egg-info
bkcharts
bkcharts-0.2-py3.7.egg-info
blaze
blaze-0.11.3-py3.7.egg-info
bleach
bleach-3.0.2-py3.7.egg-info
bokeh
bokeh-1.0.2.dist-info
boto
boto-2.49.0-py3.7.egg-info
bottleneck
Bottleneck-1.2.1-py3.7.egg-info
bs4
cachetools
cachetools-3.0.0.dist-info
caffe2
certifi
certifi-2018.11.29-py3.7.egg-info
cffi
cffi-1.11.5.dist-info
_cff

In [ ]:
!mv /usr/local/lib/python3.5/site-packages/folium  /usr/local/lib/python3.5/dist-packages/folium

In [9]:
import imp
imp.find_module('folium')

(None, '/usr/local/lib/python3.5/dist-packages/folium', ('', '', 5))

In [29]:
sys.path.insert(0,'/usr/local/lib/python3.5/site-packages')

In [30]:
sys.path.insert(0,'/usr/local/lib/python3.5/dist-packages')

In [31]:
sys.path

['/usr/local/lib/python3.5/dist-packages',
 '/usr/local/lib/python3.5/site-packages',
 '/home/ms_rachel_shalom',
 '/opt/anaconda3/lib/python37.zip',
 '/opt/anaconda3/lib/python3.7',
 '/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/opt/anaconda3/lib/python3.7/site-packages',
 '/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython']

In [41]:
imp.find_module('folium')

(None, '/usr/local/lib/python3.5/dist-packages/folium', ('', '', 5))

In [45]:
!ls /usr/local/lib/python3.5/dist-packages/folium-0.8.3+39.g8595240-py3.5.egg/folium

features.py  map.py	  raster_layers.py  vector_layers.py
folium.py    plugins	  templates	    _version.py
__init__.py  __pycache__  utilities.py


/usr/local/lib/python3.5/dist-packages
fatal: destination path 'folium' already exists and is not an empty directory.
Cloning into 'jinja'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 10563 (delta 9), reused 21 (delta 9), pack-reused 10538
Receiving objects: 100% (10563/10563), 4.44 MiB | 0 bytes/s, done.
Resolving deltas: 100% (7072/7072), done.


In [23]:
!jupyter kernelspec list

Available kernels:
  python2    /usr/local/share/jupyter/kernels/python2
  python3    /usr/local/share/jupyter/kernels/python3


In [26]:
sys.executable

'/opt/anaconda3/bin/python'

In [32]:
!ls /usr



bin   games	    lib       python   share	TensorRT-Release-Notes.pdf
data  graphsurgeon  licenses  samples  src	uff
doc   include	    local     sbin     targets
